> **Note**: This is a version of the practical without the code. I will use this during class and fill in the code live. I encourage students to type along with me since it can help with understanding and retention of the content. You will always have the full version as a reference.

# Gravity and the isostatic state of the lithosphere

**Goal:** Determine the relationship between observations (gravity disturbances and topography) and the isostatic state of the lithosphere.

The isostatic state of the lithosphere (under an Airy or Pratt model) is tied to its density structure. As such, we would expect observations of gravity disturbances to yield some knowlegde on isostasy since they are sensitive to variations in density. In this practical, we will work global observations of gravity and topography, determine a physical/mathematical relationship (a model) between these data and isostasy, test whether our model fits the observations, and reason about why is does or does not. 

## Import the required libraries

Load the required software to load the data, make maps, and perform calculations.

In [ ]:
# For arrays and linear algebra
import numpy as np
# To load and manipulate grids
import xarray as xr
# To make figures and maps
import matplotlib.pyplot as plt
# To add cartographic projections to our maps
import cartopy.crs as ccrs

## The data

**Download:** [global-gravity.nc](https://github.com/leouieda/lithosphere/raw/main/data/global-gravity.nc) (place it in the same folder as this notebook)

The gravity and topography data we will use are stored in a [netCDF](https://en.wikipedia.org/wiki/NetCDF) file, which is a very common standard for storing and distributing gridded data. It's used throughout the Earth, ocean, and climate sciences and can be read an manipulated with most software platforms. 

The gravity grid was generated from the [EIGEN-6C4 spherical harmonic model](https://doi.org/10.5880/icgem.2015.1) through the [ICGEM webservice](http://icgem.gfz-potsdam.de/home). It includes two grids: the gravity disturbance calculated with the WGS84 ellipsoid and the Bouguer gravity disturbance calculated using a topography density of 2670 kg/m³. The data for both grids was generated on top of the Earth's surface (the water surface in the oceans and topography on land).

The topography grid is a downsampled version of [ETOPO1](https://doi.org/10.7289/V5C8276M) and includes a version smoothed using a Gaussian filter with 1 degree width (~100 km).

## Load the data

The best way to work with netCDF grids in Python is with the [xarray](http://xarray.pydata.org/en/stable/) library. It allows us to easily load the data, containing all of the grids and coordinates in the same variable (an `xarray.Dataset`). It also contains many utilities for doing calculations and plotting the grids with matplotlib.

Putting the `xarray.Dataset` at the of a cell will display a nice summary of the data and metadata in the notebook. Click on the arrows to expand variables and the icons on the right to see associated metadata with each data grid.

We can access individual data grids (called `xarray.DataArray`s) like so:

## Plotting the grids

`xarray.DataArray` objects provide a `.plot` method that will display the grid data in a map, including setting appropriate colorbars and labels. This is the quickest way to visualize our data. You can provide quite a few argument to `plot` to customize the figure. Here, we will specify the figure size (in inches) so that we can actually see some things.

Notice that the colorbar is centered and all axis are labeled automatically! 🚀

The map we just plotted is showing the **gravity disturbance** for the entire globe. The most noteworthy thing about this map is that disturbance values are very small except for a few places. These generally coincide with **convergent plate boundaries** and **volcanic island chains**, both places where regional isostasy (Airy and Pratt models) are known to not be applicable. 

The gravity disturbance is a good indicator of the overall isostatic state of a region. By looking at the map, we can get a **qualitative idea of regional isostasy** but we have no good way of comparing different regions quantitavely. For that, we will need to create a model of what observations would be when the Airy or Pratt models of isostasy apply. We can then assess **how well this model fits our data**.

---

## **Your turn!**

Plot the other grids in our dataset: Bouguer gravity disturbance, topography, and smoothed topography. Try using the `"terrain"` colormap for the topography.

---

## Caveat about plotting latitude/longitude coordinates

In the plots above, the pseudo-color maps have evenly spaced longitude and latitude axis. This is troublesome because the Earth is not flat. Longitude lines get closer together as latitude increases towards the poles, which creates distortions in the map. Distortions are unavoidable but we can control them better by using [cartographic projections](https://en.wikipedia.org/wiki/Map_projection). This is handled by the [Cartopy library](https://scitools.org.uk/cartopy/docs/latest/). 

As an example, let's plot one of the global grids using the [sinusoidal projection](https://en.wikipedia.org/wiki/Sinusoidal_projection).

**Warning:** Many global projections (Robinon, Mollweide, etc) can be extremely slow on large grids like these. 

## Why the disturbance is small

Let's assume that the Earth's crust is a single homogenous layer (constant density) and it's in Airy-type isostatic equilibrium. In this case, the only density constrasts left between the real Earth and the reference Earth will be:

1. The contrast between crust and mantle due to the undulation of the real Moho.
1. The effect of the topographic masses outside of the reference Earth.
1. The contrast bewtween ocean water in the real Earth and crust on the reference Earth.

<img style="width: 700px; margin: 10px auto;" src="https://github.com/leouieda/lithosphere/raw/main/practicals/images/gravity-interpretation-isostasy.svg">
<p style="text-align: center;">
Figure: Sketch of the density constrasts that generate gravitational attraction in the gravity disturbance measured at a point.
</p>

We can approximate the gravitational attraction of topographic masses with a thickness $h$ and density $\rho$ by using the Bouguer slab equation:

$$ g_{h} \approx 2 \pi G \rho_c h $$

in which $G$ is the gravitational constant and $\rho_c$ is the density of the crust. This is gives us a simple equation to calculate the effect of topography or the Moho undulation (by using a different $\rho$ and $h$). 

Under the assumption of Airy isostasy, the thickness of the root (the Moho undulation $r$) is related to $h$ and the density of the crust $\rho_c$ and upper mantle $\rho_m$:

$$ r = \dfrac{\rho_c}{\rho_m - \rho_c} h $$

The gravitational effect of this root can be approximated with the Bouguer slab equation as well:

$$ g_{r} \approx 2 \pi G (\rho_c - \rho_m) r $$

Substituting the equation for $r$ from the Airy model we get:

$$ g_{r} -\approx 2 \pi G \rho_c h $$

In other words, **the gravitational effects of the crustal root (Moho undulation) should cancel out with the effect of topography** under the assumption of Airy isostasy. 


## On your own: Pratt isostasy

Workout what the gravity disturbance would be in the case of Pratt isostasy, where compensation is achieved by changing the density instead of adding a root. Follow the same steps that we did for Airy isostasy and you should be able to arrive at a similar solution.

## Modelling gravity vs topography

When we perform a Bouguer (or topographic) correction, we remove the gravitational effect of the topography and oceans. So our Bouguer gravity disturbance in this simplified state should only include the effect of the Moho undulation. In other words, the observed Bouguer disturbances should obey the following equation:

$$ \delta g_{bg} = 2 \pi G (\rho_c - \rho_m) r $$

or equivalently 

$$ \delta g_{bg} = - 2 \pi G \rho_c h $$

In the oceans, we can apply the same principles to arrive at

$$ \delta g_{bg} = - 2 \pi G (\rho_c - \rho_w) h $$

Both equations state that under the assumption of regional isostatic equilibrium, the observed **Bouguer gravity disturbances are linear functions of the topography and bathymetry**. We also predict that the slope of the curve is different on the contents and oceans.

This is a relationship that we can easily calculate given the topographic heights and the density of the crust and oceans. We can then compare the predictions of our model to observed data.

Now that we have a model, we can use it to make predictions of observations.

Let's plot the values of our predicted and observed gravity versus observed topography.

The plot shows that the model agrees relatively well with our data. In general, there are two linear trends and their slopes change when crossing 0 height.

But the fit is not very good for the deepest parts of the oceans. Remember that these depths occur at subduction zones, where we don't expect regional isostasy to hold.

Now, what happens if we use a version of the topography that was smoothed to remove wavelengths less that approximately 100 km? 

The fit is much better to this data! Notice that very deep oceanic areas were removed by the smoothing, as well as the smaller oceanic islands.

This is an indication that **regional Airy-type isostasy describes well the state of the lithosphere at long wavelengths**. Short wavelength topographic features must be supported by different mechanisms.

## The isostatic state of Japan

One of the advantages of xarray is that is lets us slice grids using physical coordinates instead of indices. For example, let's cut a section of our data that corresponds to Japan (where the subduction zone might cause interesting features in the data).

Now we can plot the smaller grid using a projection and coastlines. In this case, let's go with the [Lambert azimuthal equal-area projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) centered in Japan.

And we can repeat our modelling calculations for this smaller section of the data using our function from before.

As expected, the model fits the data better if we use a smoothned version of the topography. This removes at lot of the short-wavelength signal from the subduction zone. But even then, the fit to oceanic areas is still poor. This can be explained by the downward flexure of the lithosphere at the subduction zone. The bathymetry we observe there is **not due to isostasy** but instead due to the force of the Pacific subcting underneath Japan.

---

## **Your turn!**

In groups:

1. Choose a region of the Earth (between 10 - 20 degrees wide/tall) and make a slice of the global grids.
1. Calculate the predicted Airy gravity vs topography function and compare against the available data. 
1. Discuss **why** this region is or is not in isostatic equilibrium based on our modelling results. 
1. Share you results and main discussion points with the class.

---

## On your own: Explore different regions 

Now you have the data and code to model gravity vs topography in cases with isostaic equilibrium. Use this freedom to investigate the isostatic state of different geologic settings. Do some exploring on your own time and try to connect what you find with concepts you've learned or researched about the geologic setting of each region.

Here are some places you can try:

* Mid-ocean ridges
* Rift valleys
* Mountain ranges
* Large depressions in the continents
* Cratonic areas at the center of large continents
* Subduction zones

Use the cells below to write your code (add more if needed with the `+` button in the toolbar).